# Import Libraries

In [1]:
from bs4 import BeautifulSoup

In [11]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
from tqdm import tqdm

In [3]:
import requests

In [4]:
import json

In [5]:
import numpy as np

# Create All Classes Needed

In [21]:
#produkTas merupakan kelas yang akan diolah menjadi suatu relasi yang berisikan informasi-informasi terkait produk, 
#seperti nama produk, harga, rating, informasi ada atau tidaknya diskon, kondisi stok, serta kategori yang direpresentasikan
#dalam ID bertipe integer
class produkTas:
    def __init__(self, name, price, rating, isdiscountapplied, kondisistok, idkategori):
        self.name = name
        self.price = price
        self.rating = rating
        self.isdiscountapplied = isdiscountapplied
        self.kondisistok = kondisistok
        self.idkategori = idkategori
    def outputObject(self):
        print(f'''
        Name : {self.name}
        Price : {self.price}
        Rating : {self.rating}
        isdiscountapplied : {self.isdiscountapplied}
        Kondisi Stok : {self.kondisistok}
        ID_Kategori : {self.idkategori}
        ''')

In [22]:
#Kategori merupakan kelas yang akan diolah menjadi suatu relasi yang berisikan informasi-informasi terkait kategori produk
#Atribut-atribut pada produk ini terdiri atas "idkategori" yang diambil dari nama URL setiap halaman kategori produk 
#dan "kategori" yang diambil dari title pada setiap halaman katalog dengan kategori yang berbeda

class kategori:
    def __init__(self, idkategori, kategori):
        self.idkategori = idkategori
        self.kategori = kategori
    def outputObject2(self):
        print(f'''
        ID_Kategori : {self.idkategori}
        Kategori : {self.kategori}
        ''')

In [23]:
#produkberdiskon merupakan kelas yang akan diolah menjadi suatu relasi yang berisikan informasi terkait 
#produk-produk yang memiliki diskon
class produkberdiskon:
    def __init__(self, name, pricebeforediscount, priceafterdiscount, discount):
        self.name = name
        self.pricebeforediscount = pricebeforediscount
        self.priceafterdiscount = priceafterdiscount
        self.discount = discount
    def outputObject3(self):
        print(f'''
        Name : {self.name}
        PriceBeforeDiscount : {self.pricebeforediscount}
        PriceAfterDiscount : {self.priceafterdiscount}
        Discount : {self.discount}
        ''')

# Create Functions

In [40]:
#function for transforming a textual formatted price data into an integer
def fixPriceType(price):
    price = price.replace('Rp ', '')
    price = price.replace(',', '')
    price = int(price)
    return price
#function for webscraping
def scraping(list_of_objects, id_kategori):
    try:
        url = 'https://zakwoowstyle.com/categories/' + id_kategori
        header = {'user-agent' : 'Google Chrome Version 102.0.5005.63 (Official Build) (64-bit); imaazzahra38@gmail.com'}
        req = requests.get(url, headers = header)
        soup = BeautifulSoup(req.content, 'html.parser')
        print("Scraping " + url)
        #setiap produk memiliki class yang sama, yang menaungi berbagai informasi produk di subclass-nya
        list_tas = soup.findAll('div', class_='flex xs6 sm4')
        list_desc = []
        infostok = []
        inforating = []
        isDiskonExist = False
        #building 'kategori' class
        kat = soup.find('title').get_text()
        kat = kat.replace(' | Zakwoowstyle', '')
        kategoriObj = kategori(int(id_kategori), kat)
        #kategoriObj.outputObject2()
        #list_of_objects[2] berisikan list of kategoriObj
        list_of_objects[2].append(kategoriObj)
        #scraping product info and building produkTas class
        
        for tas in tqdm(list_tas):
            tas_props = tas.findAll('div', class_="lines-1")

            soldout = tas.select('div[class*="soldout"]')
            if (len(soldout) == 0):
                infostok.append('Stok masih ada')
            else:
                infostok.append('Stok habis')
            tas_props = tas.findAll('div', class_="lines-1")
            #Cleaning the data
            for desc in tqdm(tas_props):
                y = desc.get_text().strip()
                y = y.replace("\n", "")
                list_desc.append(y)
            #scraping "rating" data
            ratingclass = tas.findAll('div', class_="primary--text")
            for r in tqdm(ratingclass):
                rt = r.get_text()
                rt = rt.strip()
                if len(rt) == 1:
                    rt = float(rt)
                    inforating.append(rt)
                elif (len(rt) == 0):
                    inforating.append(0.0)
        #data preprocessing
        p = list_desc
        for i in p:
            if 'Diskon' in i:
                p.remove(i)
        #gather and transform all datas scraped into a list that contains elements with the same pattern
        halflist = len(p)/3
        split_list = np.array_split(p, halflist)
        infoproduk = []
        for array in split_list:
            infoproduk.append(list(array))
        for i in range(len(infoproduk)):
            #Memperbaiki data yang mengandung tanda petik tunggal (apostrope) pada string menjadi ''
            #sehingga sesuai dengan standar SQL
            if ("Zakwoow's" in infoproduk[i][0]):
                infoproduk[i][0] = infoproduk[i][0].replace("Zakwoow's", "Zakwoow''s")
            #if there is only one price data, it means the product doesn't offer any discount
            if infoproduk[i][2] == '':
                infoproduk[i][1] = fixPriceType(infoproduk[i][1])
                price = infoproduk[i][1]
                isDiskonExist = False
            else:
                infoproduk[i][2] = fixPriceType(infoproduk[i][2])
                infoproduk[i][1] = fixPriceType(infoproduk[i][1])
                isDiskonExist = True
                price = infoproduk[i][2]
                #Mengisi class produkBerdiskon
                besarDiscount = ((infoproduk[i][1] - infoproduk[i][2])/infoproduk[i][1])*100
                withDiscount = produkberdiskon(infoproduk[i][0], infoproduk[i][1], infoproduk[i][2], besarDiscount)
                #withDiscount.outputObject3()
                list_of_objects[1].append(withDiscount)
            #Membuat object bertipe produkTas
            newProduk = produkTas(infoproduk[i][0], price, inforating[i], isDiskonExist, infostok[i], int(id_kategori))
            #newProduk.outputObject()
            list_of_objects[0].append(newProduk)
            time.sleep(2)
    except Exception as e:
        pass


In [41]:
list_of_objects = [[],[],[]]

In [42]:
#Scraping 1/3 
scraping(list_of_objects, '199036')
scraping(list_of_objects, '199045')

Scraping https://zakwoowstyle.com/categories/199036


100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 48.45it/s]


Scraping https://zakwoowstyle.com/categories/199045


100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 192.01it/s]


In [43]:
#Scraping 2/3
scraping(list_of_objects, '198660')
scraping(list_of_objects, '199040')

Scraping https://zakwoowstyle.com/categories/198660


100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3013.87it/s]

100%|██████████| 1/1 [00:00<00:00, 1011.89it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3016.76it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 1001.74it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 4007.94it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 53.09it/s]


Scraping https://zakwoowstyle.com/categories/199040


100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 112.02it/s]


In [44]:
#Scraping 3/3
scraping(list_of_objects, '199044')
scraping(list_of_objects, '199047')
scraping(list_of_objects, '199048')

Scraping https://zakwoowstyle.com/categories/199044


100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 999.83it/s]

100%|██████████| 3/3 [00:00<00:00, 2989.53it/s]

100%|██████████| 1/1 [00:00<00:00, 1008.49it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 4011.77it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3037.15it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:00<00:00, 59.49it/s]


Scraping https://zakwoowstyle.com/categories/199047


100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 1012.87it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 2013.59it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|

Scraping https://zakwoowstyle.com/categories/199048


100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 1004.38it/s]

100%|██████████| 3/3 [00:00<00:00, 3002.37it/s]

100%|██████████| 1/1 [00:00<00:00, 1001.98it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3007.39it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 2998.07it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<?, ?it/s]

100%|█████████

In [45]:
print(len(list_of_objects[0])) #berisikan data setiap produk yang disimpan dalam relasi produkTas
print(len(list_of_objects[1])) #berisikan data harga produk yang dikenai diskon, disimpan dalam relasi produkBerdiskon
print(len(list_of_objects[2])) #berisikan data kategori produk

63
17
7


In [46]:
for i in range(len(list_of_objects[0])):
    list_of_objects[0][i].outputObject()


        Name : Mini Aira Bag by Zakwoowstyle
        Price : 90000
        Rating : 0.0
        isdiscountapplied : False
        Kondisi Stok : Stok habis
        ID_Kategori : 199036
        

        Name : Clearly Backpack Black
        Price : 85000
        Rating : 5.0
        isdiscountapplied : False
        Kondisi Stok : Stok habis
        ID_Kategori : 199036
        

        Name : Posca Backpack by Zakwoowstyle
        Price : 50000
        Rating : 0.0
        isdiscountapplied : True
        Kondisi Stok : Stok habis
        ID_Kategori : 199036
        

        Name : Belova 3 Ways Mini Backpack by Zakwoowstyle
        Price : 85000
        Rating : 0.0
        isdiscountapplied : False
        Kondisi Stok : Stok habis
        ID_Kategori : 199036
        

        Name : Clearly Backpack
        Price : 85000
        Rating : 0.0
        isdiscountapplied : False
        Kondisi Stok : Stok habis
        ID_Kategori : 199036
        

        Name : Aeera Backpack 

In [47]:
for i in range(len(list_of_objects[1])):
    list_of_objects[1][i].outputObject3()


        Name : Posca Backpack by Zakwoowstyle
        PriceBeforeDiscount : 150000
        PriceAfterDiscount : 50000
        Discount : 66.66666666666666
        

        Name : Kizzy Health Kit by Zakwoowstyle
        PriceBeforeDiscount : 99000
        PriceAfterDiscount : 40000
        Discount : 59.59595959595959
        

        Name : Apricya Slingbag by Zakwoowstyle
        PriceBeforeDiscount : 85000
        PriceAfterDiscount : 59500
        Discount : 30.0
        

        Name : Animal Retro Bag by Zakwoowstyle
        PriceBeforeDiscount : 85000
        PriceAfterDiscount : 59500
        Discount : 30.0
        

        Name : Cameela Handbag by Zakwoowstyle
        PriceBeforeDiscount : 115000
        PriceAfterDiscount : 80500
        Discount : 30.0
        

        Name : [Doodle Series] Digital Watch by Zakwoowstyle
        PriceBeforeDiscount : 230000
        PriceAfterDiscount : 184000
        Discount : 20.0
        

        Name : Two Tone Baby Corduroy Shi

In [48]:
for i in range(len(list_of_objects[2])):
    list_of_objects[2][i].outputObject2()


        ID_Kategori : 199036
        Kategori : Backpack
        

        ID_Kategori : 199045
        Kategori : Waistbag
        

        ID_Kategori : 198660
        Kategori : Wallet
        

        ID_Kategori : 199040
        Kategori : Slingbag
        

        ID_Kategori : 199044
        Kategori : Totebag/ Handbag
        

        ID_Kategori : 199047
        Kategori : Fashion
        

        ID_Kategori : 199048
        Kategori : Others
        


# DUMP JSON DATA

In [49]:
with open('produkTas.json', 'w') as f:
    json.dump([obj.__dict__ for obj in list_of_objects[0]], f, indent=4)

In [50]:
with open('produkBerdiskon.json', 'w') as f:
    json.dump([obj.__dict__ for obj in list_of_objects[1]], f, indent=4)

In [51]:
with open('kategori.json', 'w') as f:
    json.dump([obj.__dict__ for obj in list_of_objects[2]], f, indent=4)